In [16]:

#We take advantage that the board is mod 10 and so the die can be mod 10
cnt = 0
def rollUntilStop(loc1, loc2, thresholdScore):
    die=0
    numDice = 3
    turn = 0
    
    score1 = 0
    score2 = 0
    cnt = 0
    while(max(score1,score2) < thresholdScore):
        numSpaces = (die+numDice)*(die+numDice+1)/2 - die*(die+1)/2
        die = (die+numDice)%10
        if turn == 0:
            loc1 = (loc1+numSpaces)%10
            score1 += loc1 if loc1 != 0 else 10
        else:
            loc2 = (loc2+numSpaces)%10
            score2 += loc2 if loc2 != 0 else 10
        turn = 1 - turn
        cnt += numDice
        if int(cnt/numDice) % 1000 == 0:
            print("Rolled {} sets".format(int(cnt/num)))
        
    return loc1, score1, loc2, score2, cnt

loc1, score1, loc2, score2, cnt = rollUntilStop(4,8, 1000)
print("Game ends with player 1 at {} with score {} player 2 at {} with score {} and {} dice rolled".format(loc1, score1, loc2, score2, cnt))
print("End Value", min(score1,score2)*cnt)


Game ends with player 1 at 0.0 with score 1000.0 player 2 at 3.0 with score 745.0 and 993 dice rolled
End Value 739785.0


In [44]:
#Could look at state space and iterate that way, e.g. 10 spaces for player 1 X 10 spaces for player 2 X 21 scores for player 1 X 21 scores for player 2,
#Or just simulate

#We do simplify by realizing that we are summing over random variables and no matter how many ways a 5 is rolled, its still a 5
#Sums of 3 dice
#1,1,1
#1,1,2
#1,1,3
#...
#3,3,3
def getCombinationDict():
    C={}
    for i in range(1,4):
        for j in range(1,4):
            for k in range(1,4):
                s = i+j+k
                if s in C:
                    C[s] += 1
                else:
                    C[s] = 1
    return C


CDict = getCombinationDict()

print(CDict)
threshold = 21

def evalDieRoll(loc, score, die):
    val = (loc+die)%10
    newloc = 10 if val == 0 else val
    return newloc, score+newloc

def iterateUniverses(loc1, loc2, score1, score2, turn, threshold = 1):
    if score1 >= threshold:
        return 1, 0
    if score2 >= threshold:
        return 0, 1
    s1,s2 = 0,0
    if turn == 0:
        for i, count in CDict.items():
            newloc, newscore = evalDieRoll(loc1,score1, i)
            a,b = iterateUniverses(newloc, loc2, newscore, score2, 1, threshold)
            s1 += count*a
            s2 += count*b
    else:
        for i, count in CDict.items():
            newloc, newscore = evalDieRoll(loc2,score2, i)
            a,b = iterateUniverses(loc1, newloc, score1, newscore, 0, threshold)
            s1 += count*a
            s2 += count*b
    
            
    return s1, s2        


s1, s2 = iterateUniverses(4,8,0,0,0,21)
print(max(s1,s2))


{3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}
444356092776315
